In [3]:
#All needed libraries for the analysis
import pandas as pd
import requests, time
from datetime import datetime as dt
import warnings
from pandas.core.common import SettingWithCopyWarning

In [4]:
rawData = pd.read_json("scans.json")
rawData.head(10)

,scan,date,name,surface,start,end
0,1596752162134,2020-08-08 05:15:54.366,Weight Room,fitness,52200,54000
1,1596752162134,2020-08-08 05:15:54.949,Weight Room,fitness,52200,54000
2,1596752162134,2020-08-08 05:15:55.485,Weight Room,fitness,52200,54000
3,1596752162134,2020-08-08 05:15:56.013,Weight Room,fitness,52200,54000
4,1596752162134,2020-08-08 05:15:56.543,Weight Room,fitness,52200,54000
5,1596752162134,2020-08-08 05:15:57.076,Weight Room,fitness,52200,54000
6,1596752162134,2020-08-08 05:15:57.630,Weight Room,fitness,52200,54000
7,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 3),lane,45000,46800
8,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,46800,48600
9,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,48600,50400


In [5]:
rawData = rawData[rawData.date > pd.Timestamp("2020-03-01")]
#I dropped all time before lockdown and COVID-19 procedures started, as differences in gym attendence and procedures may 
#interfere with data analysis by including non-represenative data

rawData.head(10)

,scan,date,name,surface,start,end
0,1596752162134,2020-08-08 05:15:54.366,Weight Room,fitness,52200,54000
1,1596752162134,2020-08-08 05:15:54.949,Weight Room,fitness,52200,54000
2,1596752162134,2020-08-08 05:15:55.485,Weight Room,fitness,52200,54000
3,1596752162134,2020-08-08 05:15:56.013,Weight Room,fitness,52200,54000
4,1596752162134,2020-08-08 05:15:56.543,Weight Room,fitness,52200,54000
5,1596752162134,2020-08-08 05:15:57.076,Weight Room,fitness,52200,54000
6,1596752162134,2020-08-08 05:15:57.630,Weight Room,fitness,52200,54000
7,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 3),lane,45000,46800
8,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,46800,48600
9,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,48600,50400


In [6]:
def gym_loc(name):
    if name.startswith("Baby Pool"):
        return "Baby Pool"
    if name.startswith("South Pool"):
        return "South Pool"
    if name.startswith("Weight"):
        return "Lifts"
    if name.startswith("North Pool"):
        return name.split("(")[1]
        #we only need the lanes from the North Pool slots 
    return name
#this function will define the gym location of the different areas. Only the North Pool is needed for this particular analysis
#and setting everything to be the same will make it easier to clean out data.

rawData["location"] = rawData["name"].apply(gym_loc)
#Making a new column for the gym locations using the function


rawData["location"] = rawData['location'].str.replace(")","")
#dropping off the last parenthesis in the lane rows, as right now it reads "lane 3)"

rawData.head(10)

,scan,date,name,surface,start,end,location
0,1596752162134,2020-08-08 05:15:54.366,Weight Room,fitness,52200,54000,Lifts
1,1596752162134,2020-08-08 05:15:54.949,Weight Room,fitness,52200,54000,Lifts
2,1596752162134,2020-08-08 05:15:55.485,Weight Room,fitness,52200,54000,Lifts
3,1596752162134,2020-08-08 05:15:56.013,Weight Room,fitness,52200,54000,Lifts
4,1596752162134,2020-08-08 05:15:56.543,Weight Room,fitness,52200,54000,Lifts
5,1596752162134,2020-08-08 05:15:57.076,Weight Room,fitness,52200,54000,Lifts
6,1596752162134,2020-08-08 05:15:57.630,Weight Room,fitness,52200,54000,Lifts
7,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 3),lane,45000,46800,lane 3
8,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,46800,48600,Baby Pool
9,1596752162134,2020-08-08 05:15:58.331,Baby Pool 1,lane,48600,50400,Baby Pool


In [7]:
rawData.drop(rawData[rawData['location'] == "Baby Pool"].index, inplace = True)
rawData.drop(rawData[rawData['location'] == "South Pool"].index, inplace = True)
#We're dropping the South and Baby pool data because we need only the north pool. I'm sure there is a way to put this on one line
#but I couldn't find it :( 

rawData.drop(rawData[rawData['start'] < 36000].index, inplace = True)
#dropping all data for times before 10AM because we don't want to the gym before 10 AM. We use 36000 because right now
#the start column is in the seconds since 12 AM so 60 seconds * 60 minutes = 3600 seconds in an hour. 3600 * 10 = seconds
#equivalent to 10AM = 36000

rawData.head(10)

,scan,date,name,surface,start,end,location
0,1596752162134,2020-08-08 05:15:54.366,Weight Room,fitness,52200,54000,Lifts
1,1596752162134,2020-08-08 05:15:54.949,Weight Room,fitness,52200,54000,Lifts
2,1596752162134,2020-08-08 05:15:55.485,Weight Room,fitness,52200,54000,Lifts
3,1596752162134,2020-08-08 05:15:56.013,Weight Room,fitness,52200,54000,Lifts
4,1596752162134,2020-08-08 05:15:56.543,Weight Room,fitness,52200,54000,Lifts
5,1596752162134,2020-08-08 05:15:57.076,Weight Room,fitness,52200,54000,Lifts
6,1596752162134,2020-08-08 05:15:57.630,Weight Room,fitness,52200,54000,Lifts
7,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 3),lane,45000,46800,lane 3
10,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 6),lane,50400,52200,lane 6
11,1596752162134,2020-08-08 05:15:58.331,North Pool (lane 3),lane,50400,52200,lane 3


In [8]:
rawData['start'] = pd.to_datetime(rawData['start'], unit = 's')
rawData['end'] = pd.to_datetime(rawData['end'], unit = 's')
#converting to a datetime object to get the actual, readable human time

rawData['start'] = rawData['start'].dt.strftime('%H:%M')
rawData['end'] = rawData['end'].dt.strftime('%H:%M')
#trimming unnecessary information, we only need the hour and the minutes

rawData['date'] = rawData['date'].dt.strftime('%A')
#we will also need the day of the week for data analyst, this will grab that for us


rawData.head(10)

,scan,date,name,surface,start,end,location
0,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
1,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
2,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
3,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
4,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
5,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
6,1596752162134,Saturday,Weight Room,fitness,14:30,15:00,Lifts
7,1596752162134,Saturday,North Pool (lane 3),lane,12:30,13:00,lane 3
10,1596752162134,Saturday,North Pool (lane 6),lane,14:00,14:30,lane 6
11,1596752162134,Saturday,North Pool (lane 3),lane,14:00,14:30,lane 3


In [10]:
warnings.simplefilter(action="ignore", category = SettingWithCopyWarning)
#We're going to ignore the SettingWithCopyWarning because I do want a copy, not a view
#Also this appears to be a problem with pandas. The below code would shoot out SettingWithCopyWarning despite documentation
#Saying that the bug where df2['column'] = df1['column1'] + df1['column2'] was fixed as that shouldn't spit a warning
#but here we are.

data = rawData[['location']]
#For the data dateframe, we first need to grab location
data['pkey'] = rawData['date'] + " "+ rawData['start'] + "-" + rawData['end'] 
#This will be our primary key for the dateframe. To do analysis we need the day of the week, start, end, and location
#location info is here twice, but its use will become apparent soon

data.head(10)

,location,pkey
0,Lifts,Saturday 14:30-15:00
1,Lifts,Saturday 14:30-15:00
2,Lifts,Saturday 14:30-15:00
3,Lifts,Saturday 14:30-15:00
4,Lifts,Saturday 14:30-15:00
5,Lifts,Saturday 14:30-15:00
6,Lifts,Saturday 14:30-15:00
7,lane 3,Saturday 12:30-13:00
10,lane 6,Saturday 14:00-14:30
11,lane 3,Saturday 14:00-14:30


In [11]:
gymWindow = data[['pkey','location']]
#We will make a seperate dataframe for just the gym/lifting data to avoid having the pool data clog up the screen
gymWindow.drop(gymWindow[gymWindow['location'] != "Lifts"].index, inplace = True)
#Dropping everything that is not pertraining to the gym and lifting weights
gymWindow["count"] = gymWindow['location']
#creating a new column called count, copying location to give it a value for now. We will modify count later 
gymWindow.head(10)

,pkey,location,count
0,Saturday 14:30-15:00,Lifts,Lifts
1,Saturday 14:30-15:00,Lifts,Lifts
2,Saturday 14:30-15:00,Lifts,Lifts
3,Saturday 14:30-15:00,Lifts,Lifts
4,Saturday 14:30-15:00,Lifts,Lifts
5,Saturday 14:30-15:00,Lifts,Lifts
6,Saturday 14:30-15:00,Lifts,Lifts
322,Saturday 14:30-15:00,Lifts,Lifts
323,Saturday 14:30-15:00,Lifts,Lifts
324,Saturday 14:30-15:00,Lifts,Lifts


In [12]:
gymWindow.groupby('pkey').count()[['count']]
#We will group by the primary key, and then count only the count column. As a result, only the count column will have its values
#replaced

#note: the data from the bot is broken right now and is only picking up Saturday and Sunday for some reason, will be fixed
#later

,count
pkey,
Saturday 14:30-15:00,56


In [13]:
poolWindow = data[['pkey','location']]
poolWindow.drop(poolWindow[poolWindow['location'] == "Lifts"].index, inplace = True)
poolWindow["count"] = poolWindow['location']
#doing the same thing as we did with the gym data to the pool data

poolWindow.head(10)

,pkey,location,count
7,Saturday 12:30-13:00,lane 3,lane 3
10,Saturday 14:00-14:30,lane 6,lane 6
11,Saturday 14:00-14:30,lane 3,lane 3
13,Saturday 14:30-15:00,lane 6,lane 6
14,Saturday 14:30-15:00,lane 4,lane 4
15,Saturday 14:30-15:00,lane 2,lane 2
17,Saturday 15:30-16:00,lane 2,lane 2
20,Saturday 16:30-17:00,lane 6,lane 6
21,Saturday 16:30-17:00,lane 4,lane 4
22,Saturday 16:30-17:00,lane 3,lane 3


In [14]:
poolWindow.groupby(['location', 'pkey']).count()[['count']]

count
location pkey                       
lane 1   Saturday 16:30-17:00    350
         Saturday 17:00-17:30    245
         Saturday 17:30-18:00    350
         Saturday 18:00-18:30    350
         Saturday 18:30-19:00    350
         Sunday 16:30-17:00       63
         Sunday 17:30-18:00       63
         Sunday 18:00-18:30       63
         Sunday 18:30-19:00       63
lane 2   Saturday 14:30-15:00    213
         Saturday 15:30-16:00    350
         Saturday 18:00-18:30    350
         Saturday 18:30-19:00    350
         Sunday 15:30-16:00       44
         Sunday 18:00-18:30       63
         Sunday 18:30-19:00       63
lane 3   Saturday 12:30-13:00     25
         Saturday 14:00-14:30    350
         Saturday 16:30-17:00    300
         Saturday 17:00-17:30    299
         Saturday 17:30-18:00    350
         Saturday 18:00-18:30    350
         Saturday 19:30-20:00    350
         Sunday 17:30-18:00       63
         Sunday 18:00-18:30       63
         Sunday 19:30-20:00       63
lane 4   Saturday 14:30-15:00    350
         Saturday 16:30-17:00    322
         Saturday 17:00-17:30    350
         Saturday 17:30-18:00    315
         Saturday 18:00-18:30    315
         Saturday 18:30-19:00    350
         Saturday 19:00-19:30    350
         Saturday 19:30-20:00    350
         Sunday 14:30-15:00       63
         Sunday 17:00-17:30       63
         Sunday 18:30-19:00       63
         Sunday 19:00-19:30       63
         Sunday 19:30-20:00       63
lane 5   Saturday 17:00-17:30    222
         Saturday 17:30-18:00    222
         Saturday 18:00-18:30    350
         Saturday 18:30-19:00    350
         Saturday 19:00-19:30    350
         Saturday 19:30-20:00    350
         Sunday 18:00-18:30       63
         Sunday 18:30-19:00       63
         Sunday 19:00-19:30       63
         Sunday 19:30-20:00       63
lane 6   Saturday 14:00-14:30    159
         Saturday 14:30-15:00    159
         Saturday 16:30-17:00    350
         Sunday 16:30-17:00       63